## Imports

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import sys
import numpy as np
import os

## Utilising GPU using Pytorch

In [2]:
# cpu-gpu
a = torch.randn((3, 4))
print(a.device)

device = torch.device("cuda")
a = a.to(device)
print(a.device)

# a more generic code
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cpu
cuda:0


In [3]:
torch.cuda.is_available() 

True

In [4]:
!nvidia-smi

Sat Sep 17 19:40:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.43.04    Driver Version: 515.43.04    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:09:00.0 Off |                  N/A |
|  0%   39C    P2   109W / 350W |  11104MiB / 24576MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Dataset and Transforms

In [5]:
train_transform = transforms.Compose([
  transforms.RandomCrop(32, padding=4),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
test_transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_dset = torchvision.datasets.CIFAR10(root="data/", train=True, transform=train_transform, download=True)
test_dset = torchvision.datasets.CIFAR10(root="data/", train=False, transform=test_transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
print(f"# of train samples: {len(train_dset)}")
print(f"# of test samples: {len(test_dset)}")

# of train samples: 50000
# of test samples: 10000


In [7]:
train_loader = DataLoader(train_dset, batch_size=100, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dset, batch_size=100, shuffle=False, num_workers=2)

In [8]:
print(f"# of train batches: {len(train_loader)}")
print(f"# of test batches: {len(test_loader)}")

# of train batches: 500
# of test batches: 100


In [9]:
print("sample i/o sizes")
data = next(iter(train_loader))
img, target = data
print(f"input size: {img.shape}")
print(f"output size: {target.shape}")

sample i/o sizes
input size: torch.Size([100, 3, 32, 32])
output size: torch.Size([100])


## LeNet

In [10]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
    self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
    # TODO: missing input feature size
    self.fc1   = nn.Linear(16*5*5, 120)
    self.fc2   = nn.Linear(120, 84)
    # TODO: missing output feature size
    self.fc3   = nn.Linear(84, 10)
    self.activ = nn.ReLU()

  # TODO: add maxpool operation of given kernel size
  # https://pytorch.org/docs/stable/nn.functional.html
  def pool(self, x, kernel_size=2):
    out = F.max_pool2d(x,kernel_size)
    return out

  def forward(self, x):
    out = self.activ(self.conv1(x))
    out = self.pool(out)
    out = self.activ(self.conv2(out))
    out = self.pool(out)

    # TODO: flatten
    out = out.view(out.size(0),-1)
    out = self.activ(self.fc1(out))
    out = self.activ(self.fc2(out))
    out = self.fc3(out)
    return out

## VGG

In [11]:
class VGG(nn.Module):
  CONFIGS = {
      "vgg11": [64, "pool", 128, "pool", 256, 256, "pool", 512, 512, "pool", 512, 512, "pool"],
      "vgg13": [64, 64, "pool", 128, 128, "pool", 256, 256, "pool", 512, 512, "pool", 512, 512, "pool"],
      "vgg16": [64, 64, "pool", 128, 128, "pool", 256, 256, 256, "pool", 512, 512, 512, "pool", 512, 512, 512, "pool"],
      "vgg19": [64, 64, "pool", 128, 128, "pool", 256, 256, 256, 256, "pool", 512, 512, 512, 512, "pool", 512, 512, 512, 512, "pool"],
  }
  def __init__(self, cfg):
    super(VGG, self).__init__()
    # TODO: missing input dimension
    in_dim = 3
    layers = []
    for layer in self.CONFIGS[cfg]:
        if layer == "pool":
            # TODO: add maxpool module of given kernel size, stride (here 2 each)
            # https://pytorch.org/docs/stable/nn.html
            maxpool = nn.MaxPool2d(kernel_size=2,stride=2)
            layers.append(maxpool)
        else:
            # TODO: add sequential module consisting of convolution (kernel size = 3, padding = 1), batchnorm, relu
            # https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html?highlight=sequential#torch.nn.Sequential
            block = nn.Sequential(
                nn.Conv2d(in_channels=in_dim,out_channels=layer,kernel_size=3,padding=1),
                nn.BatchNorm2d(layer),
                nn.ReLU()
            )
            layers.append(block)
            in_dim = layer
    # TODO: add average pool to collapse spatial dimensions
    avgpool = nn.AvgPool2d(kernel_size=1)
    layers.append(avgpool)
    self.layers = nn.Sequential(*layers)
    # TODO: missing output features
    self.fc = nn.Linear(512,10)

  def forward(self, x):
    out = self.layers(x)
    # TODO: flatten
    out = out.reshape(out.size(0),-1)
    out = self.fc(out)
    return out

## ResNet

In [85]:
class BasicBlock(nn.Module):
  expansion = 1

  def __init__(self, in_dim, dim, stride=1):
    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_dim, dim, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(dim)
    self.conv2 = nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(dim)
    self.activ = nn.ReLU()

    self.shortcut = nn.Identity()
    # TODO: missing condition for parameterized shortcut connection (hint: when input and output dimensions don't match - both spatial, feature)
    if (in_dim!=dim):
        # TODO: add sequential module consisting of 1x1 convolution (given stride, bias=False), batchnorm
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_dim,dim,kernel_size=1,stride=stride,bias=False),
            nn.BatchNorm2d(dim)
        )
      
  def forward(self, x):
    out = self.activ(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    # TODO: missing residual connection
    out += self.shortcut(x)
    out = self.activ(out)
    return out


class Bottleneck(nn.Module):
  expansion = 4

  def __init__(self, in_dim, dim, stride=1):
    super(Bottleneck, self).__init__()
    self.conv1 = nn.Conv2d(in_dim, dim, kernel_size=1, bias=False)
    self.bn1 = nn.BatchNorm2d(dim)
    self.conv2 = nn.Conv2d(dim, dim, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(dim)
    self.conv3 = nn.Conv2d(dim, self.expansion * dim, kernel_size=1, bias=False)
    self.bn3 = nn.BatchNorm2d(self.expansion*dim)
    self.activ = nn.ReLU()

    self.shortcut = nn.Identity()
    # TODO: missing condition for parameterized shortcut connection (hint: when input and output dimensions don't match - both spatial, feature)
    if (in_dim!=dim*self.expansion):
        # TODO: add sequential module consisting of 1x1 convolution (given stride, bias=False), batchnorm
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_dim,dim*self.expansion,kernel_size=1,stride=stride,bias=False),
            nn.BatchNorm2d(self.expansion*dim)
        )

  def forward(self, x):
    out = self.activ(self.bn1(self.conv1(x)))
    out = self.activ(self.bn2(self.conv2(out)))
    out = self.bn3(self.conv3(out))
    # TODO: missing residual connection
    out += self.shortcut(x)
    out = self.activ(out)
    return out


class ResNet(nn.Module):
  CONFIGS = {
      "resnet18": (BasicBlock, [2, 2, 2, 2]),
      "resnet34": (BasicBlock, [3, 4, 6, 3]),
      "resnet50": (Bottleneck, [3, 4, 6, 3]),
      "resnet101": (Bottleneck, [3, 4, 23, 3]),
      "resnet152": (Bottleneck, [3, 8, 36, 3]),
  }
  def __init__(self, cfg):
    super(ResNet, self).__init__()
    block, num_blocks = self.CONFIGS[cfg]
    self.in_dim = 64
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._make_layer(block, 64, num_blocks[0],stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
    self.activ = nn.ReLU()
    # TODO: missing output features
    self.linear = nn.Linear(512*block.expansion, 10)

  def _make_layer(self, block, dim, num_blocks, stride):
    strides = [stride] + [1]*(num_blocks-1)    
    layers = []
    for stride in strides: 
        # TODO: create layers within block
        layer =block(
            self.in_dim,dim,stride
        )
        layers.append(layer)
        # TODO: update in_dim based on block output size
        self.in_dim = dim*block.expansion
    return nn.Sequential(*layers)

  def forward(self, x):
    out = self.activ(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    # TODO: average pool and flatten
    out = F.avg_pool2d(out,out.shape[2])
    out = out.view(out.size(0),-1)

    out = self.linear(out)
    return out

## Utility functions (can ignore)

In [13]:
def pbar(p=0, msg="", bar_len=20):
    sys.stdout.write("\033[K")
    sys.stdout.write("\x1b[2K" + "\r")
    block = int(round(bar_len * p))
    text = "Progress: [{}] {}% {}".format(
        "\x1b[32m" + "=" * (block - 1) + ">" + "\033[0m" + "-" * (bar_len - block),
        round(p * 100, 2),
        msg,
    )
    print(text, end="\r")
    if p == 1:
        print()


class AvgMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.metrics = {}

    def add(self, batch_metrics):
        if self.metrics == {}:
            for key, value in batch_metrics.items():
                self.metrics[key] = [value]
        else:
            for key, value in batch_metrics.items():
                self.metrics[key].append(value)

    def get(self):
        return {key: np.mean(value) for key, value in self.metrics.items()}

    def msg(self):
        avg_metrics = {key: np.mean(value) for key, value in self.metrics.items()}
        return "".join(["[{}] {:.5f} ".format(key, value) for key, value in avg_metrics.items()])

## Training

In [14]:
def train(model, optim, lr_sched=None, epochs=20, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"), criterion=None, metric_meter=None, out_dir="out/"):
  model.to(device)
  best_acc = 0
  for epoch in range(epochs):
    model.train()
    metric_meter.reset()
    for indx, (img, target) in enumerate(train_loader):
      # TODO: send to device (cpu or gpu)
      img = img.to(device)
      target = target.to(device)

      # TODO: missing forward pass
      out = model(img)
      loss = criterion(out, target)
      # TODO: missing backward, parameter update
      optim.zero_grad()
      loss.backward()
      optim.step()
      metric_meter.add({"train loss": loss.item()})
      pbar(indx / len(train_loader), msg=metric_meter.msg())
    pbar(1, msg=metric_meter.msg())

    model.eval()
    metric_meter.reset()
    for indx, (img, target) in enumerate(test_loader):
      # TODO: send to device (cpu or gpu)
      img = img.to(device)
      target = target.to(device)

      # TODO: missing forward pass
      out = model(img)
      loss = criterion(out, target)
      # TODO: compute accuracy
      acc = (out.argmax(1) == target).type(torch.float).sum().item()

      metric_meter.add({"test loss": loss.item(), "test acc": acc})
      pbar(indx / len(test_loader), msg=metric_meter.msg())
    pbar(1, msg=metric_meter.msg())
    
    test_metrics = metric_meter.get()
    if test_metrics["test acc"] > best_acc:
      print(
          "\x1b[33m"
          + f"test acc improved from {round(best_acc, 5)} to {round(test_metrics['test acc'], 5)}"
          + "\033[0m"
      )
      best_acc = test_metrics['test acc']
      torch.save(model.state_dict(), os.path.join(out_dir, "best.ckpt"))
    lr_sched.step()

## Run Experiments

In [15]:
def run_experiment(model_name="lenet", model_cfg=None, epochs=20):
  if model_name == "lenet":
    model = LeNet()
  elif model_name == "vgg":
    model = VGG(model_cfg)
  elif model_name == "resnet":
    model = ResNet(model_cfg)
  else:
    raise NotImplementedError()
  optim = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=5e-4)
  lr_sched = torch.optim.lr_scheduler.CosineAnnealingLR(optim, T_max=epochs)
  criterion = nn.CrossEntropyLoss()
  metric_meter = AvgMeter()
  out_dir = f"{model_name}_{model_cfg}"
  os.makedirs(out_dir, exist_ok=True)
  train(model, optim, lr_sched, epochs=epochs, criterion=criterion, metric_meter=metric_meter, out_dir=out_dir)

In [16]:
run_experiment(model_name="lenet")

Progress: [===================>] 100% [train loss] 1.99768  
Progress: [===================>] 100% [test loss] 1.85006 [test acc] 28.98000  
test acc improved from 0 to 28.98
Progress: [===================>] 100% [train loss] 1.91818  
Progress: [===================>] 100% [test loss] 1.80415 [test acc] 32.53000  
test acc improved from 28.98 to 32.53
Progress: [===================>] 100% [train loss] 1.86331  
Progress: [===================>] 100% [test loss] 1.79053 [test acc] 32.07000  
Progress: [===================>] 100% [train loss] 1.83782  
Progress: [===================>] 100% [test loss] 1.76359 [test acc] 30.90000  
Progress: [===================>] 100% [train loss] 1.80683  
Progress: [===================>] 100% [test loss] 1.72777 [test acc] 36.10000  
test acc improved from 32.53 to 36.1
Progress: [===================>] 100% [train loss] 1.77354  
Progress: [===================>] 100% [test loss] 1.65982 [test acc] 38.66000  
test acc improved from 36.1 to 38.66
Progress

In [17]:
run_experiment(model_name="vgg", model_cfg="vgg11")

Progress: [===================>] 100% [train loss] 2.49837   
Progress: [===================>] 100% [test loss] 2.11753 [test acc] 18.64000  
test acc improved from 0 to 18.64
Progress: [===================>] 100% [train loss] 1.82412  
Progress: [===================>] 100% [test loss] 1.71281 [test acc] 32.83000  
test acc improved from 18.64 to 32.83
Progress: [===================>] 100% [train loss] 1.49933  
Progress: [===================>] 100% [test loss] 1.32061 [test acc] 52.28000  
test acc improved from 32.83 to 52.28
Progress: [===================>] 100% [train loss] 1.18352  
Progress: [===================>] 100% [test loss] 1.00651 [test acc] 64.18000  
test acc improved from 52.28 to 64.18
Progress: [===================>] 100% [train loss] 0.99579  
Progress: [===================>] 100% [test loss] 1.13784 [test acc] 59.84000  
Progress: [===================>] 100% [train loss] 0.88512  
Progress: [===================>] 100% [test loss] 0.97336 [test acc] 67.41000  
test 

In [18]:
run_experiment(model_name="vgg", model_cfg="vgg13")

Progress: [===================>] 100% [train loss] 2.44108   
Progress: [===================>] 100% [test loss] 2.07746 [test acc] 20.70000  
test acc improved from 0 to 20.7
Progress: [===================>] 100% [train loss] 1.79503  
Progress: [===================>] 100% [test loss] 1.62918 [test acc] 38.93000  
test acc improved from 20.7 to 38.93
Progress: [===================>] 100% [train loss] 1.52694  
Progress: [===================>] 100% [test loss] 1.38246 [test acc] 48.90000  
test acc improved from 38.93 to 48.9
Progress: [===================>] 100% [train loss] 1.21315  
Progress: [===================>] 100% [test loss] 1.06427 [test acc] 62.47000  
test acc improved from 48.9 to 62.47
Progress: [===================>] 100% [train loss] 0.97325  
Progress: [===================>] 100% [test loss] 1.19326 [test acc] 60.26000  
Progress: [===================>] 100% [train loss] 0.81624  
Progress: [===================>] 100% [test loss] 0.75401 [test acc] 73.96000  
test acc 

In [19]:
run_experiment(model_name="vgg", model_cfg="vgg16")

Progress: [===================>] 100% [train loss] 2.49230  
Progress: [===================>] 100% [test loss] 2.24595 [test acc] 14.35000  
test acc improved from 0 to 14.35
Progress: [===================>] 100% [train loss] 1.97769  
Progress: [===================>] 100% [test loss] 2.00301 [test acc] 23.58000  
test acc improved from 14.35 to 23.58
Progress: [===================>] 100% [train loss] 1.65402  
Progress: [===================>] 100% [test loss] 1.66933 [test acc] 40.78000  
test acc improved from 23.58 to 40.78
Progress: [===================>] 100% [train loss] 1.33771  
Progress: [===================>] 100% [test loss] 1.13723 [test acc] 60.03000  
test acc improved from 40.78 to 60.03
Progress: [===================>] 100% [train loss] 1.06645  
Progress: [===================>] 100% [test loss] 1.10258 [test acc] 61.46000  
test acc improved from 60.03 to 61.46
Progress: [===================>] 100% [train loss] 0.90210  
Progress: [===================>] 100% [test loss

In [20]:
run_experiment(model_name="vgg", model_cfg="vgg19")

Progress: [===================>] 100% [train loss] 2.36738  
Progress: [===================>] 100% [test loss] 1.97781 [test acc] 21.20000  
test acc improved from 0 to 21.2
Progress: [===================>] 100% [train loss] 1.93916  
Progress: [===================>] 100% [test loss] 1.93952 [test acc] 22.01000  
test acc improved from 21.2 to 22.01
Progress: [===================>] 100% [train loss] 1.85289  
Progress: [===================>] 100% [test loss] 1.98220 [test acc] 26.42000  
test acc improved from 22.01 to 26.42
Progress: [===================>] 100% [train loss] 1.70820  
Progress: [===================>] 100% [test loss] 1.70799 [test acc] 35.32000  
test acc improved from 26.42 to 35.32
Progress: [===================>] 100% [train loss] 1.52197  
Progress: [===================>] 100% [test loss] 1.39538 [test acc] 48.63000  
test acc improved from 35.32 to 48.63
Progress: [===================>] 100% [train loss] 1.39908  
Progress: [===================>] 100% [test loss] 

In [91]:
run_experiment(model_name="resnet", model_cfg="resnet18")

Progress: [===================>] 100% [train loss] 1.85363  
Progress: [===================>] 100% [test loss] 1.50344 [test acc] 45.19000  
test acc improved from 0 to 45.19
Progress: [===================>] 100% [train loss] 1.39760  
Progress: [===================>] 100% [test loss] 1.36675 [test acc] 51.39000  
test acc improved from 45.19 to 51.39
Progress: [===================>] 100% [train loss] 1.10971  
Progress: [===================>] 100% [test loss] 1.00291 [test acc] 64.05000  
test acc improved from 51.39 to 64.05
Progress: [===================>] 100% [train loss] 0.91560  
Progress: [===================>] 100% [test loss] 1.07837 [test acc] 63.15000  
Progress: [===================>] 100% [train loss] 0.75763  
Progress: [===================>] 100% [test loss] 0.81671 [test acc] 72.21000  
test acc improved from 64.05 to 72.21
Progress: [===================>] 100% [train loss] 0.65539  
Progress: [===================>] 100% [test loss] 0.73771 [test acc] 74.90000  
test a

In [93]:
run_experiment(model_name="resnet", model_cfg="resnet50")

Progress: [===================>] 100% [train loss] 2.49105   
Progress: [===================>] 100% [test loss] 1.95167 [test acc] 22.99000  
test acc improved from 0 to 22.99
Progress: [===================>] 100% [train loss] 1.84636  
Progress: [===================>] 100% [test loss] 1.71040 [test acc] 34.28000  
test acc improved from 22.99 to 34.28
Progress: [===================>] 100% [train loss] 1.59796  
Progress: [===================>] 100% [test loss] 1.41821 [test acc] 46.98000  
test acc improved from 34.28 to 46.98
Progress: [===================>] 100% [train loss] 1.32265  
Progress: [===================>] 100% [test loss] 1.18074 [test acc] 56.99000  
test acc improved from 46.98 to 56.99
Progress: [===================>] 100% [train loss] 1.13106  
Progress: [===================>] 100% [test loss] 1.17298 [test acc] 57.31000  
test acc improved from 56.99 to 57.31
Progress: [===================>] 100% [train loss] 1.00609  
Progress: [===================>] 100% [test los

In [95]:
run_experiment(model_name="resnet", model_cfg="resnet152")

Progress: [===================>] 100% [train loss] 2.61806   
Progress: [===================>] 100% [test loss] 1.98638 [test acc] 22.47000  
test acc improved from 0 to 22.47
Progress: [===================>] 100% [train loss] 1.93957  
Progress: [===================>] 100% [test loss] 1.79311 [test acc] 31.27000  
test acc improved from 22.47 to 31.27
Progress: [===================>] 100% [train loss] 1.71470  
Progress: [===================>] 100% [test loss] 1.57397 [test acc] 42.86000  
test acc improved from 31.27 to 42.86
Progress: [===================>] 100% [train loss] 1.48099  
Progress: [===================>] 100% [test loss] 1.40066 [test acc] 48.99000  
test acc improved from 42.86 to 48.99
Progress: [===================>] 100% [train loss] 1.28673  
Progress: [===================>] 100% [test loss] 1.25056 [test acc] 54.62000  
test acc improved from 48.99 to 54.62
Progress: [===================>] 100% [train loss] 1.12041  
Progress: [===================>] 100% [test los

## Questions
- Train and report test set metrics on three model types - LeNet, VGG, ResNet. 
- Which model performs the best and why?
- Which model performs the worst and why?

- Number of the Epochs run = 20
<div align="center">

|Sl No|Model Name |Test set Accuracies in Percentage(%)|
|:--:|:-----------:|:------------------------------------:|
|1. |LeNEt |62.57|
|2.| VGG11 | 88.49|
|3.| VGG13 | 90.66|
|4.| VGG16 | 90.81|
|5.| VGG19 | 88.27|
|6.| ResNet18| 92.26|
|7.| ResNet50| 89.35|
|8.| ResNet152| 89.77|


</div>


1. The performance of LeNet is not so good. Lenet performs the worst because of shallow architecture and no better optimisation to push the accuracy.


2. Resnet performs better ,The principle on which ResNets work is to build a deeper networks compared to other plain networks and simultaneously find a optimised number of layers to negate the vanishing gradient problem.
* Note : Increased Resenet performs worst than its lower versions , so one need to find the fit properly according to the dataset and model to be used

#### Bonus Marks
Separate File (```EE21S055_Tutorial6_Bonus.ipynb```) \
1 Increased the channel from 16 to 32 
* 16->32 Accuracy Moved from 62.57% to 64.48%

2 Increased the channel from 32 to 64
* 32->64  Accuracy Moved from  64.48% to 71.33

This clearly shows that the increase in the channel number increased the accuracies 